# Sexperto.co scraping
## Made by David Bautista

In [183]:
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
import numpy as np
import sys
import time
import requests
import unicodedata
import pandas as pd
pd.set_option('max_colwidth', 1000)
import unidecode

In [78]:
def getAnswers(query):
    response = None
    ret = []
    ht = requests.get('http://54.90.49.105:8899/api/v1/proxies?https=true&limit=5')
    url = 'https://sexperto.co/pregunta/{}'.format(query)
    if(ht.status_code == 200):
        proxies = ht.json()
        proxies = [None] + proxies['proxies']
        while(len(proxies) > 0):
            try:
                print("quedan {} de 6 proxies".format(len(proxies)))
                proxy_info = proxies.pop()
                proxy = {"https": 'https://{}:{}'.format(proxy_info['ip'], proxy_info['port'])}
                time.sleep(2)
                response = requests.get(url)
                if(response.status_code == 429):
                    raise Exception("429 :/") 
                else:
                    break
            except Except as e:
                print("paila", e)
                continue
    if response != None and response.status_code == 200:
        print("response 200 :D")
        soup = BeautifulSoup(response.text, 'lxml')
        targets = soup.find_all(["span", "div"], ["masinfo", "respuesta-corta"])
        ret = [target.text for target in targets]
    return ret

In [38]:
soup = BeautifulSoup(response.text, 'lxml')

In [40]:
targets = soup.find_all(["span", "div"], ["masinfo", "respuesta-corta"])

In [68]:
file = open("sexperto_preguntas.json", 'r')
questions = json.load(file)

In [ ]:
for index, question in enumerate(questions):
    question["respuestas"] = getAnswers(question["FriendlyName"])
    print("terminada la pregunta", index)
    print("*****************************")

In [184]:
df = pd.DataFrame(questions)
for i in df:
    df[i] = df[i].apply(lambda x: re.sub(r'(<.*?>)?', "", unidecode.unidecode(str(x))))
df['respuestas'] = df["respuestas"].apply(lambda x: ". ".join(x).replace(u"\xa0", u""))

In [186]:
df[["Name", "respuestas_juntas"]].to_json("sin_tildes_sexperto.json", orient="records")

In [171]:
questions[0]

{'QuestionID': 1496,
 'FriendlyName': 'es-necesario-tomar-anticoncepcion-de-emergencia-cuando-se-utiliza-el-anillo-anticonceptivo-vaginal-nuvaring-sin-condon',
 'Name': "¿Es necesario tomar <a class='bold'>anticoncepción</a> de <a class='bold'>emergencia</a> cuando se utiliza el anillo <a class='bold'>anticonceptivo</a> <a class='bold'>vagina</a>l (NuvaRing) sin <a class='bold'>condón</a>?",
 'Count': 5050,
 'respuestas': 'Solo cuando comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de la menstruación, es necesario utilizar un método anticonceptivo adicional. Si este es tu caso y no usaste condón si sería necesario usar anticoncepción de emergencia. Te recomiendo asistir a una asesoría en anticoncepción en tu centro de atención en salud. Recuerda utilizar condón en todas las relaciones sexuales.',
 'respuestas_juntas': 'Solo cuando comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de la menstru

In [45]:
re.findall(r"\>\s*(([a-zA-Z]|[áéíóú])(\.|\,|\;)*\s*)+\s*\<", targets[0])

[]

In [50]:
targets[0].text

'El pegging es un término en inglés que se refiere a la práctica sexual en la que la mujer penetra al hombre utilizando un consolador que se sostiene con un arnés el cual se ajusta en la cintura y la cadera.\xa0'

In [ ]:
[target.text for target in targets]

In [152]:
file_out = open("sexperto_preguntas_y_respuestas.json", 'w')
json.dump(questions, file_out)

In [161]:
df = pd.DataFrame(questions)

In [168]:
df[df["QuestionID"] == 1496]["respuestas_juntas"]

0    Solo cuando comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de la menstruación, es necesario utilizar un método anticonceptivo adicional. Si este es tu caso y no usaste condón si sería necesario usar anticoncepción de emergencia. Te recomiendo asistir a una asesoría en anticoncepción en tu centro de atención en salud. Recuerda utilizar condón en todas las relaciones sexuales.
Name: respuestas_juntas, dtype: object

In [172]:
df.to_json("hola.json", orient='records')

In [136]:
df[df["QuestionID"]== "1496"]["respuestas"][0]

"['Solo cuando\\xa0 comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de la menstruación, es necesario utilizar un método anticonceptivo adicional. Si este es tu caso y no usaste condón si sería necesario usar anticoncepción de emergencia.\\xa0 Te recomiendo asistir a una asesoría en anticoncepción en tu centro de atención en salud. Recuerda utilizar condón en todas las relaciones sexuales.']"

In [175]:
file_pr = open("hola.json", 'r')
js = json.load(file_pr)

In [176]:
js[0]

{'QuestionID': 1496,
 'FriendlyName': 'es-necesario-tomar-anticoncepcion-de-emergencia-cuando-se-utiliza-el-anillo-anticonceptivo-vaginal-nuvaring-sin-condon',
 'Name': "¿Es necesario tomar <a class='bold'>anticoncepción</a> de <a class='bold'>emergencia</a> cuando se utiliza el anillo <a class='bold'>anticonceptivo</a> <a class='bold'>vagina</a>l (NuvaRing) sin <a class='bold'>condón</a>?",
 'Count': 5050,
 'respuestas': ['Solo cuando\xa0 comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de la menstruación, es necesario utilizar un método anticonceptivo adicional. Si este es tu caso y no usaste condón si sería necesario usar anticoncepción de emergencia.\xa0 Te recomiendo asistir a una asesoría en anticoncepción en tu centro de atención en salud. Recuerda utilizar condón en todas las relaciones sexuales.'],
 'respuestas_juntas': 'Solo cuando comienzas a usar el anillo anticonceptivo vaginal en un día diferente a los primeros cinco días de 

In [187]:
df

,QuestionID,FriendlyName,Name,Count,respuestas,respuestas_juntas
0,1496,es-necesario-tomar-anticoncepcion-de-emergencia-cuando-se-utiliza-el-anillo-anticonceptivo-vaginal-nuvaring-sin-condon,?Es necesario tomar anticoncepcion de emergencia cuando se utiliza el anillo anticonceptivo vaginal (NuvaRing) sin condon?,5050,"S. o. l. o. . c. u. a. n. d. o. . c. o. m. i. e. n. z. a. s. . a. . u. s. a. r. . e. l. . a. n. i. l. l. o. . a. n. t. i. c. o. n. c. e. p. t. i. v. o. . v. a. g. i. n. a. l. . e. n. . u. n. . d. i. a. . d. i. f. e. r. e. n. t. e. . a. . l. o. s. . p. r. i. m. e. r. o. s. . c. i. n. c. o. . d. i. a. s. . d. e. . l. a. . m. e. n. s. t. r. u. a. c. i. o. n. ,. . e. s. . n. e. c. e. s. a. r. i. o. . u. t. i. l. i. z. a. r. . u. n. . m. e. t. o. d. o. . a. n. t. i. c. o. n. c. e. p. t. i. v. o. . a. d. i. c. i. o. n. a. l. .. . S. i. . e. s. t. e. . e. s. . t. u. . c. a. s. o. . y. . n. o. . u. s. a. s. t. e. . c. o. n. d. o. n. . s. i. . s. e. r. i. a. . n. e. c. e. s. a. r. i. o. . u. s. a. r. . a. n. t. i. c. o. n. c. e. p. c. i. o. n. . d. e. . e. m. e. r. g. e. n. c. i. a. .. . T. e. . r. e. c. o. m. i. e. n. d. o. . a. s. i. s. t. i. r. . a. . u. n. a. . a. s. e. s. o. r. i. a. . e. n. . a. n. t. i. c. o. n. c. e. p. c. i. o. n. ...","Solo cuando comienzas a usar el anillo anticonceptivo vaginal en un dia diferente a los primeros cinco dias de la menstruacion, es necesario utilizar un metodo anticonceptivo adicional. Si este es tu caso y no usaste condon si seria necesario usar anticoncepcion de emergencia. Te recomiendo asistir a una asesoria en anticoncepcion en tu centro de atencion en salud. Recuerda utilizar condon en todas las relaciones sexuales."
1,3804,es-posible-que-mi-pareja-quede-embarazada-si-eyacule-fuera-de-la-vagina-antes-de-la-penetracion-y-ademas-tomo-el-anticonceptivo-de-emergencia,?Es posible que mi pareja quede embarazada si eyacule fuera de la vagina antes de la penetracion y ademas tomo el anticonceptivo de emergencia?,5044,"A. u. n. q. u. e. . d. u. r. a. n. t. e. . l. a. . p. e. n. e. t. r. a. c. i. o. n. . v. a. g. i. n. a. l. . l. u. e. g. o. . d. e. . l. a. . e. y. a. c. u. l. a. c. i. o. n. . p. u. e. d. e. n. . q. u. e. d. a. r. . p. e. q. u. e. n. a. s. . c. a. n. t. i. d. a. d. e. s. . d. e. . s. e. m. e. n. ,. . s. i. . u. s. a. r. o. n. . u. n. . a. n. t. i. c. o. n. c. e. p. t. i. v. o. . o. . p. i. l. d. o. r. a. . d. e. . e. m. e. r. g. e. n. c. i. a. ,. . e. s. . m. u. y. . p. o. c. o. . p. r. o. b. a. b. l. e. . q. u. e. . s. e. . p. r. e. s. e. n. t. e. . u. n. . e. m. b. a. r. a. z. o. ,. . e. s. p. e. c. i. a. l. m. e. n. t. e. . s. i. . t. u. . p. a. r. e. j. a. . l. o. . t. o. m. o. . e. n. . l. a. s. . s. i. g. u. i. e. n. t. e. s. . 7. 2. . a. . 1. 2. 0. . h. o. r. a. s. . l. u. e. g. o. . d. e. . l. a. . r. e. l. a. c. i. o. n. . s. e. x. u. a. l. .. . E. s. t. e. . m. e. t. o. d. o. . e. s. . a. l. t. a. m. e. n. t. e. . e. ...","Aunque durante la penetracion vaginal luego de la eyaculacion pueden quedar pequenas cantidades de semen, si usaron un anticonceptivo o pildora de emergencia, es muy poco probable que se presente un embarazo, especialmente si tu pareja lo tomo en las siguientes 72 a 120 horas luego de la relacion sexual. Este metodo es altamente efectivo para la prevencion de un embarazo no planeado, pero esta efectividad depende de que se tome en el tiempo indicado, ademas no debe ser usado como metodo de planificacion regular. En todo caso, te recomiendo usar condon siempre en tus relaciones sexuales, no solo para prevenir embarazos sino tambien para prevenir infecciones de transmision sexual.. Para mayor informacion te invitamos a consultar los siguientes contenidos de Sexperto:?Cuales son los riesgos de tomar la ""pastilla del dia despues"" en mas de una ocasion?"
2,510,la-anticoncepcion-de-emergencia-solo-me-la-pueden-vender-con-formula-medica,?La anticoncepcion de emergencia solo me la pueden vender con formula medica?,4037,"\n. N. o. ,. . l. a. . a